# Exercícios

Sua equipe  está trabalhando na auditoria da cota parlamentar para o período de 2008 a 2019. O objetivo final da sua análise é apontar possíveis irregularidades nos reembolsos, identificar solicitações suspeitas, deputados que devem ser investigados e partidos com maior índice de irregularidades. Construa uma análise utilizando dados e justificativas plausíveis possa ser utilizada na auditoria.

Escolha **três** categorias de cota parlamentar para realizar a análise. A narrativa com os dados e pensamento analítico sobre a categoria e os valores fazem parte da avaliação, que devem responder obrigatoriamente as seguintes perguntas **para cada categoria** analisada:

* Quais solicitações de reembolso são suspeitas ?
* Das solicitações suspeitas, escolha uma para analisar mais afundo e justifique o porque devemos indiciar o parlamentar ?
* Quais parlamentares devem ser investigados (top 5)?
* Eleja um parlamentar para ser premiado por boa conduta. Justifique sua escolha com dados. 

Escolha **um** parlamentar de sua preferência (alguém que você votou se lembrar). Analise os gastos específicos desse parlamentar e responda às seguintes perguntas:

* O parlamentar gasta a verba completa ou tem um gasto moderado ?
* Qual o perfil de gastos do parlamentar, quais categorias o parlamentar mais gasta a verba ?
* Existe algum reembolso suspeito em relação às categorias analisadas ?
* Analisando as empresas prestadoras de serviços (cnpj_cpf), existe algo suspeito ? 

**TENTE APLICAR A METODOLOGIA CRISP-DM**

# Iniciando a Análise

## Leitura do Dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_cota_parlamentar = pd.read_csv('dadosBrutos\CEAPS_2008_2019.csv')

In [3]:
df_cota_parlamentar

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO
0,2008,9,ADA MELLO,"Contratação de consultorias, assessorias, pesq...",NaN,NaN,NaN,NaN,NaN,12351.52
1,2008,9,ADA MELLO,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,386.60
2,2008,10,ADA MELLO,"Contratação de consultorias, assessorias, pesq...",NaN,NaN,NaN,NaN,NaN,12351.52
3,2008,10,ADA MELLO,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,2610.68
4,2008,11,ADA MELLO,"Contratação de consultorias, assessorias, pesq...",NaN,NaN,NaN,NaN,NaN,12351.52
...,...,...,...,...,...,...,...,...,...,...
268894,2019,1,ZEZÉ PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",09.296.295/0001-60,Azul,O954UL,19/01/2019,"Companhia Aérea: AZUL, Localizador: O954UL. Pa...",769.29
268895,2019,1,ZEZÉ PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",09.296.295/0001-60,Azul,G9Z36V,24/01/2019,"Companhia Aérea: AZUL, Localizador: G9Z36V. Pa...",310.62
268896,2019,1,ZEZÉ PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",07.575.651/0001-59,GOL,WNQI4A,29/01/2019,"Companhia Aérea: GOL, Localizador: WNQI4A. Pas...",1168.10
268897,2019,1,ZEZÉ PERRELLA,"Passagens aéreas, aquáticas e terrestres nacio...",33.937.681/0001-78,LATAM,NCCPIM,23/01/2019,"Companhia Aérea: LATAM, Localizador: NCCPIM. P...",772.05


In [4]:
df_cota_parlamentar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268899 entries, 0 to 268898
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ANO                268899 non-null  int64  
 1   MES                268899 non-null  int64  
 2   SENADOR            268899 non-null  object 
 3   TIPO_DESPESA       268899 non-null  object 
 4   CNPJ_CPF           265755 non-null  object 
 5   FORNECEDOR         265755 non-null  object 
 6   DOCUMENTO          246750 non-null  object 
 7   DATA               265755 non-null  object 
 8   DETALHAMENTO       154368 non-null  object 
 9   VALOR_REEMBOLSADO  268897 non-null  float64
dtypes: float64(1), int64(2), object(7)
memory usage: 20.5+ MB


In [5]:
df_cota_parlamentar.describe()

,ANO,MES,VALOR_REEMBOLSADO
count,268899.000000,268899.000000,268897.000000
mean,2014.099413,6.651040,930.433800
std,3.022301,3.282404,2453.968535
min,2008.000000,1.000000,-243.400000
25%,2012.000000,4.000000,99.700000
50%,2014.000000,7.000000,251.970000
75%,2017.000000,9.000000,900.000000
max,2019.000000,12.000000,270000.000000


# Agregamento por tipo de despesa

In [6]:
df_tipo_despesa_info = df_cota_parlamentar.groupby("TIPO_DESPESA")["VALOR_REEMBOLSADO"].agg(["count", np.std, np.min, np.max, np.sum, np.mean, np.median])

In [7]:
df_tipo_despesa_info

,count,std,amin,amax,sum,mean,median
TIPO_DESPESA,,,,,,,
"Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.",51070,1588.392262,0.01,18793.95,47922809.61,938.374968,335.675
"Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos.",22073,1017.514499,0.01,40000.00,9806573.89,444.279160,121.850
"Contratação de consultorias, assessorias, pesquisas, trabalhos técnicos e outros serviços de apoio ao exercício do mandato parlamentar",9827,6523.676345,-243.40,150000.00,37193016.49,3784.778314,1500.000
Divulgação da atividade parlamentar,10048,7249.372000,0.60,270000.00,31828000.34,3167.595575,1500.000
"Locomoção, hospedagem, alimentação, combustíveis e lubrificantes",115533,1611.214386,0.01,66730.00,64056430.15,554.442715,114.600
"Passagens aéreas, aquáticas e terrestres nacionais",59495,715.221197,-9.00,84000.00,56388198.82,947.780466,800.390
Serviços de Segurança Privada,851,6269.709108,24.00,28922.72,2995828.11,3520.362056,520.000


Visualizando do maior para o menor número de ocorrências

In [8]:
df_tipo_despesa_info.sort_values(by="count", ascending=False)

,count,std,amin,amax,sum,mean,median
TIPO_DESPESA,,,,,,,
"Locomoção, hospedagem, alimentação, combustíveis e lubrificantes",115533,1611.214386,0.01,66730.00,64056430.15,554.442715,114.600
"Passagens aéreas, aquáticas e terrestres nacionais",59495,715.221197,-9.00,84000.00,56388198.82,947.780466,800.390
"Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.",51070,1588.392262,0.01,18793.95,47922809.61,938.374968,335.675
"Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos.",22073,1017.514499,0.01,40000.00,9806573.89,444.279160,121.850
Divulgação da atividade parlamentar,10048,7249.372000,0.60,270000.00,31828000.34,3167.595575,1500.000
"Contratação de consultorias, assessorias, pesquisas, trabalhos técnicos e outros serviços de apoio ao exercício do mandato parlamentar",9827,6523.676345,-243.40,150000.00,37193016.49,3784.778314,1500.000
Serviços de Segurança Privada,851,6269.709108,24.00,28922.72,2995828.11,3520.362056,520.000


# Escolha das categorias da Análise

1°) Locomoção, hospedagem, alimentação, combustíveis e lubrificantes.

    Por ter o maior número de ocorrências.
    
2°) Serviços de Segurança Privada.

    Por ter o menor número de ocorrências.
    
3°) Divulgação da atividade parlamentar.

    Por ter o maior desvio padrão.

# Primeira Análise

## Locomoção, hospedagem, alimentação, combustíveis e lubrificantes.

In [9]:
# Preparação do data frame
df_loc = df_cota_parlamentar[df_cota_parlamentar["TIPO_DESPESA"] == "Locomoção, hospedagem, alimentação, combustíveis e lubrificantes"]

df_loc.head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO
1,2008,9,ADA MELLO,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,386.60
3,2008,10,ADA MELLO,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,2610.68
5,2008,11,ADA MELLO,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,2615.63
7,2008,12,ADA MELLO,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,4943.47
9,2008,2,ADELMIR SANTANA,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,1151.13


In [10]:
df_loc["VALOR_REEMBOLSADO"].describe()

count    115533.000000
mean        554.442715
std        1611.214386
min           0.010000
25%          50.000000
50%         114.600000
75%         209.030000
max       66730.000000
Name: VALOR_REEMBOLSADO, dtype: float64

Agora procurando o parlamentar que teve o maior reembolso nesse tipo de despesa.

In [11]:
df_loc.loc[df_loc["VALOR_REEMBOLSADO"].idxmax()]

ANO                                                               2019
MES                                                                  3
SENADOR                                                DAVI ALCOLUMBRE
TIPO_DESPESA         Locomoção, hospedagem, alimentação, combustíve...
CNPJ_CPF                                            06.180.439/0001-20
FORNECEDOR                               AEROTOP TÁXI AÉREO LTDA - EPP
DOCUMENTO                                                          766
DATA                                                        12/03/2019
DETALHAMENTO                                                       NaN
VALOR_REEMBOLSADO                                              66730.0
Name: 251545, dtype: object

Agora buscando os maiores 100 reembolsos nessa categoria.

In [12]:
df_loc_outliers = df_loc.nlargest(100, "VALOR_REEMBOLSADO")
df_loc_outliers

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO
251545,2019,3,DAVI ALCOLUMBRE,"Locomoção, hospedagem, alimentação, combustíve...",06.180.439/0001-20,AEROTOP TÁXI AÉREO LTDA - EPP,766,12/03/2019,NaN,66730.00
16669,2009,12,ROMEU TUMA,"Locomoção, hospedagem, alimentação, combustíve...",02.296.299/0001-07,REALI TAXI AÉREO LTDA,2792,23/12/2009,NaN,42400.00
2090,2008,11,ROMEU TUMA,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,41239.32
2343,2008,8,VALDIR RAUPP,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,38250.00
13022,2009,11,MAURO FECURY,"Locomoção, hospedagem, alimentação, combustíve...",01.097.044/0001-44,OPERADORA SÃO PAULO RENAISSANCE LTDA,00176486,09/11/2009,NaN,37346.22
...,...,...,...,...,...,...,...,...,...,...
141694,2014,1,VALDIR RAUPP,"Locomoção, hospedagem, alimentação, combustíve...",08.727.561/0001-07,AP RENT A CAR & REPRESENTAÇÕES,000496,30/01/2014,Aluguel de veículo para utilização no desempen...,16000.00
248635,2019,1,ÂNGELA PORTELA,"Locomoção, hospedagem, alimentação, combustíve...",07.442.073/0001-82,REAL TUR TRANSPORTE E SERVIÇOS LTDO -ME,00000103,02/01/2019,NaN,16000.00
1077,2008,12,JAYME CAMPOS,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,15988.50
1498,2008,12,MÃO SANTA,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,15977.94


### Solicitação para investigar mais a fundo

A terceira maior solicitação de reembolso, não informa o nome do fornecedor, detalhamento e data. 

In [63]:
df_loc.loc[2090]

ANO                                                               2008
MES                                                                 11
SENADOR                                                     ROMEU TUMA
TIPO_DESPESA         Locomoção, hospedagem, alimentação, combustíve...
CNPJ_CPF                                                           NaN
FORNECEDOR                                                         NaN
DOCUMENTO                                                          NaN
DATA                                                               NaN
DETALHAMENTO                                                       NaN
VALOR_REEMBOLSADO                                             41239.32
Name: 2090, dtype: object

### Top 5 parlamentares para serem investigados

In [14]:
df_loc_investigacao = df_loc.groupby(["SENADOR"])["VALOR_REEMBOLSADO"].sum().sort_values(ascending=False)
df_loc_investigacao = pd.DataFrame(df_loc_investigacao, columns=["VALOR_REEMBOLSADO"])
df_loc_investigacao

,VALOR_REEMBOLSADO
SENADOR,
CIRO NOGUEIRA,2423362.67
FLEXA RIBEIRO,1487410.74
VICENTINHO ALVES,1449320.62
HUMBERTO COSTA,1365448.64
JOÃO DURVAL,1325686.69
...,...
STYVENSON VALENTIM,636.00
RAIMUNDO LIRA,573.36
IZALCI LUCAS,535.27


In [15]:
df_loc_null = df_loc.loc[:, df_loc.columns != "SENADOR"].isnull().groupby(df_loc["SENADOR"]).sum()
df_loc_null["TOTAL_NULO"] = df_loc_null.sum(axis=1).sort_values(ascending=False)
df_loc_null.drop(["ANO", "MES", "TIPO_DESPESA", "CNPJ_CPF", "FORNECEDOR", "DOCUMENTO", "DATA",
                 "DETALHAMENTO", "VALOR_REEMBOLSADO"], axis=1, inplace=True)
df_loc_null.sort_values("TOTAL_NULO", ascending=False, inplace=True)
df_loc_null

,TOTAL_NULO
SENADOR,
JOÃO CAPIBERIBE,3276
PAULO BAUER,2762
MÁRIO COUTO,2401
MARCELO CRIVELLA,2200
FLÁVIO ARNS,2164
...,...
ODACIR SOARES,0
ELIZIANE GAMA,0
GIVAGO TENÓRIO,0


In [16]:
df_loc_investigacao_final = pd.merge(df_loc_investigacao, df_loc_null, on="SENADOR").sort_values(["VALOR_REEMBOLSADO", "TOTAL_NULO"], ascending=[False,False])
df_loc_investigacao_final.describe()

,VALOR_REEMBOLSADO,TOTAL_NULO
count,2.540000e+02,254.000000
mean,2.521907e+05,300.448819
std,3.344850e+05,477.743448
min,1.000000e+02,0.000000
25%,3.264845e+04,27.500000
50%,1.040381e+05,129.000000
75%,3.435855e+05,352.500000
max,2.423363e+06,3276.000000


In [17]:
df_loc_investigacao_final

,VALOR_REEMBOLSADO,TOTAL_NULO
SENADOR,,
CIRO NOGUEIRA,2423362.67,533
FLEXA RIBEIRO,1487410.74,339
VICENTINHO ALVES,1449320.62,368
HUMBERTO COSTA,1365448.64,1744
JOÃO DURVAL,1325686.69,1535
...,...,...
STYVENSON VALENTIM,636.00,0
RAIMUNDO LIRA,573.36,3
IZALCI LUCAS,535.27,2


Deputados escolhidos:

CIRO NOGUEIRA

FLEXA RIBEIRA

VICENTINHO ALVES

HUMBERTO COSTA

JOÃO DURVAL

In [20]:
df_loc_investigacao_final.loc["EDISON LOBÃO"]

VALOR_REEMBOLSADO    435.15
TOTAL_NULO             0.00
Name: EDISON LOBÃO, dtype: float64

Edison Lobão não é o parlamentar que menos gastou na categoria mas seguindo a ordem crescente, é o primeiro que menos utilizou a cota parlamentar a ter informado todos os dados do reembolso

# Segunda Análise

## Serviços de Segurança Privada

In [21]:
df_seg = df_cota_parlamentar[df_cota_parlamentar["TIPO_DESPESA"] == "Serviços de Segurança Privada"]
df_seg.head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO
38794,2011,8,ANA RITA,Serviços de Segurança Privada,01.887.130/0001-50,ECOS SEGURANÇA ELETRONICA LTDA ME,06437,18/08/2011,NaN,100.0
38841,2011,9,ANA RITA,Serviços de Segurança Privada,07.633.895/0001-40,DIGITAL SERVIÇOS EM SEGURANÇA ELETRÔNICA LTDA...,05752,01/09/2011,NaN,150.0
38946,2011,12,ANA RITA,Serviços de Segurança Privada,01.887.130/0001-50,ECOS SEGURANÇA ELETRÔNICA LTDA-ME,07365,15/12/2011,NaN,100.0
39699,2011,12,ANÍBAL DINIZ,Serviços de Segurança Privada,06.373.173/0001-31,Inviolável R. Branco Alarmes Monitorados,0407,13/01/2012,NaN,180.0
42102,2011,6,CLÉSIO ANDRADE,Serviços de Segurança Privada,02.059.753/0001-06,Emive Patrulha 24 horas Ltda,2011/88415,15/06/2011,NaN,109.0


In [22]:
df_seg.describe()

,ANO,MES,VALOR_REEMBOLSADO
count,851.000000,851.000000,851.000000
mean,2014.698002,6.388954,3520.362056
std,2.143495,3.482436,6269.709108
min,2011.000000,1.000000,24.000000
25%,2013.000000,3.000000,189.000000
50%,2015.000000,6.000000,520.000000
75%,2016.000000,9.000000,4300.110000
max,2019.000000,12.000000,28922.720000


In [24]:
df_seg.loc[df_seg["VALOR_REEMBOLSADO"].idxmax()]

ANO                                           2016
MES                                              2
SENADOR                            FERNANDO COLLOR
TIPO_DESPESA         Serviços de Segurança Privada
CNPJ_CPF                        04.850.792/0001-44
FORNECEDOR                           Citel Service
DOCUMENTO                                     1176
DATA                                    01/02/2016
DETALHAMENTO                                   NaN
VALOR_REEMBOLSADO                         28922.72
Name: 177257, dtype: object

In [27]:
df_seg_outliers = df_seg.nlargest(100, "VALOR_REEMBOLSADO")
df_seg_outliers.head(30)

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO
177257,2016,2,FERNANDO COLLOR,Serviços de Segurança Privada,04.850.792/0001-44,Citel Service,1176,01/02/2016,NaN,28922.72
151346,2015,2,FERNANDO COLLOR,Serviços de Segurança Privada,04.850.792/0001-44,Citel Service,1030,01/02/2015,NaN,28174.24
177259,2016,3,FERNANDO COLLOR,Serviços de Segurança Privada,04.850.792/0001-44,Citel Service,1187,01/03/2016,NaN,27655.07
177267,2016,8,FERNANDO COLLOR,Serviços de Segurança Privada,04.850.792/0001-44,Citel Service,1208,01/07/2016,NaN,27655.07
177268,2016,8,FERNANDO COLLOR,Serviços de Segurança Privada,04.850.792/0001-44,Citel Service,1220,26/08/2016,NaN,27655.07
177271,2016,10,FERNANDO COLLOR,Serviços de Segurança Privada,04.850.792/0001-44,Citel Service,1228,01/10/2016,NaN,27655.07
177273,2016,11,FERNANDO COLLOR,Serviços de Segurança Privada,04.850.792/0001-44,Citel Service,1237,01/11/2016,NaN,27655.07
177274,2016,12,FERNANDO COLLOR,Serviços de Segurança Privada,04.850.792/0001-44,Citel Service,1242,01/12/2016,NaN,27655.07
204319,2017,1,FERNANDO COLLOR,Serviços de Segurança Privada,04.850.792/0001-44,Citel Service,1245,01/12/2016,NaN,27655.07
204320,2017,2,FERNANDO COLLOR,Serviços de Segurança Privada,04.850.792/0001-44,Citel Service,1251,01/05/2017,NaN,27655.07


### Solicitação para investigar mais a fundo

Fernando color, valores altos recorrentes com a mesma empresa, faltando detalhamento de alguns registros

In [29]:
df_seg_outliers.loc[177257]

ANO                                           2016
MES                                              2
SENADOR                            FERNANDO COLLOR
TIPO_DESPESA         Serviços de Segurança Privada
CNPJ_CPF                        04.850.792/0001-44
FORNECEDOR                           Citel Service
DOCUMENTO                                     1176
DATA                                    01/02/2016
DETALHAMENTO                                   NaN
VALOR_REEMBOLSADO                         28922.72
Name: 177257, dtype: object

### Top 5 parlamentares para serem investigados

In [65]:
df_seg_investigacao = df_seg.groupby(["SENADOR"])["VALOR_REEMBOLSADO"].sum().sort_values(ascending=False)
df_seg_investigacao = pd.DataFrame(df_seg_investigacao, columns=["VALOR_REEMBOLSADO"])
df_seg_investigacao

,VALOR_REEMBOLSADO
SENADOR,
FERNANDO COLLOR,1691331.81
JOSÉ AGRIPINO,361630.75
ROBERTO ROCHA,232510.51
LOBÃO FILHO,219600.00
JOÃO CAPIBERIBE,76360.94
KÁTIA ABREU,74986.68
JARBAS VASCONCELOS,40874.12
FLEXA RIBEIRO,37099.13
MAILZA GOMES,28200.00


In [43]:
df_seg_null = df_seg.loc[:, df_seg.columns != "SENADOR"].isnull().groupby(df_seg["SENADOR"]).sum()
df_seg_null["TOTAL_NULO"] = df_seg_null.sum(axis=1).sort_values(ascending=False)
df_seg_null.drop(["ANO", "MES", "TIPO_DESPESA", "CNPJ_CPF", "FORNECEDOR", "DOCUMENTO", "DATA",
                 "DETALHAMENTO", "VALOR_REEMBOLSADO"], axis=1, inplace=True)
df_seg_null.sort_values("TOTAL_NULO", ascending=False, inplace=True)
df_seg_null

,TOTAL_NULO
SENADOR,
JOSÉ AGRIPINO,78
GLADSON CAMELI,69
FERNANDO COLLOR,56
LOBÃO FILHO,38
FÁTIMA BEZERRA,29
CLÉSIO ANDRADE,27
TELMÁRIO MOTA,25
MAILZA GOMES,13
RONALDO CAIADO,10


In [44]:
df_seg_investigacao_final = pd.merge(df_seg_investigacao, df_seg_null, on="SENADOR").sort_values(["VALOR_REEMBOLSADO", "TOTAL_NULO"], ascending=[False,False])
df_seg_investigacao_final.describe()

,VALOR_REEMBOLSADO,TOTAL_NULO
count,5.000000e+01,50.000000
mean,5.991656e+04,9.660000
std,2.446479e+05,16.991727
min,4.500000e+01,0.000000
25%,7.234875e+02,0.250000
50%,3.996900e+03,4.000000
75%,1.862144e+04,9.000000
max,1.691332e+06,78.000000


In [46]:
df_seg_investigacao_final

,VALOR_REEMBOLSADO,TOTAL_NULO
SENADOR,,
FERNANDO COLLOR,1691331.81,56
JOSÉ AGRIPINO,361630.75,78
ROBERTO ROCHA,232510.51,5
LOBÃO FILHO,219600.00,38
JOÃO CAPIBERIBE,76360.94,0
KÁTIA ABREU,74986.68,5
JARBAS VASCONCELOS,40874.12,9
FLEXA RIBEIRO,37099.13,10
MAILZA GOMES,28200.00,13


Deputados escolhidos:

FERNANDO COLLOR

JOSÉ AGRIPINO

LOBÃO FILHO	

GLADSON CAMELI - valor menor mas muitos faltantes

FÁTIMA BEZERRA - valor menor mas muito faltantes

In [48]:
df_seg_investigacao_final.loc["LÍDICE DA MATA"]

VALOR_REEMBOLSADO    75.0
TOTAL_NULO            0.0
Name: LÍDICE DA MATA, dtype: float64

Lídice da Mata não é o parlamentar que menos gastou na categoria mas seguindo a ordem crescente, é o primeiro que menos utilizou a cota parlamentar a ter informado todos os dados do reembolso

# Segunda Análise

## Divulgação da atividade parlamentar

In [58]:
df_divul = df_cota_parlamentar[df_cota_parlamentar["TIPO_DESPESA"] == "Divulgação da atividade parlamentar"]
df_divul.head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO
26,2008,11,ADELMIR SANTANA,Divulgação da atividade parlamentar,NaN,NaN,NaN,NaN,NaN,15000.0
33,2008,2,ALMEIDA LIMA,Divulgação da atividade parlamentar,NaN,NaN,NaN,NaN,NaN,10500.0
64,2008,12,ALMEIDA LIMA,Divulgação da atividade parlamentar,NaN,NaN,NaN,NaN,NaN,35000.0
72,2008,3,ALOIZIO MERCADANTE,Divulgação da atividade parlamentar,NaN,NaN,NaN,NaN,NaN,6300.0
76,2008,4,ALOIZIO MERCADANTE,Divulgação da atividade parlamentar,NaN,NaN,NaN,NaN,NaN,1575.0


In [59]:
df_divul.describe()

,ANO,MES,VALOR_REEMBOLSADO
count,10048.000000,10048.000000,10048.000000
mean,2014.101413,6.927747,3167.595575
std,3.078542,3.535788,7249.372000
min,2008.000000,1.000000,0.600000
25%,2012.000000,4.000000,800.000000
50%,2014.000000,7.000000,1500.000000
75%,2017.000000,10.000000,3000.000000
max,2019.000000,12.000000,270000.000000


In [60]:
df_divul.loc[df_divul["VALOR_REEMBOLSADO"].idxmax()]

ANO                                                   2017
MES                                                     12
SENADOR                                    DAVI ALCOLUMBRE
TIPO_DESPESA           Divulgação da atividade parlamentar
CNPJ_CPF                                29.337.655/0001-31
FORNECEDOR           Start print Comunicação Visual Eireli
DOCUMENTO                                               36
DATA                                            15/03/2018
DETALHAMENTO                                           NaN
VALOR_REEMBOLSADO                                 270000.0
Name: 201299, dtype: object

In [61]:
df_divul_outliers = df_divul.nlargest(100, "VALOR_REEMBOLSADO")
df_divul_outliers.head(10)

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO
201299,2017,12,DAVI ALCOLUMBRE,Divulgação da atividade parlamentar,29.337.655/0001-31,Start print Comunicação Visual Eireli,36,15/03/2018,NaN,270000.0
174534,2016,12,DAVI ALCOLUMBRE,Divulgação da atividade parlamentar,04.953.008/0001-23,ARTE & IMAGEM GRAFICA E EDITORA EIRELI ME,202,22/03/2017,NaN,256980.0
148380,2015,12,DAVI ALCOLUMBRE,Divulgação da atividade parlamentar,38.006.631/0001-90,Gráfica e Editora Paranaíba Ltda - ME,242,29/03/2016,NaN,216780.0
138773,2014,12,PAULO PAIM,Divulgação da atividade parlamentar,07.053.387/0001-93,Algo Mais Artes Gráficas e Editora Ltda,12786,17/12/2014,"Impressão de 800 mil informativos, 30 mil esta...",162400.0
166059,2015,12,ROSE DE FREITAS,Divulgação da atividade parlamentar,28.130.334/0001-07,Gráfica Samorini Ltda Epp,269,29/03/2016,NaN,134100.0
244762,2018,12,ROSE DE FREITAS,Divulgação da atividade parlamentar,05.461.408/0001-84,Grafitusa S/A,9681,29/03/2019,NaN,103900.0
80571,2012,12,LINDBERGH FARIAS,Divulgação da atividade parlamentar,00.971.215/0001-50,3 GRAPH GRAFICA E EDITORA LTDA,00000998,11/12/2012,NaN,88245.0
169207,2015,12,WELLINGTON FAGUNDES,Divulgação da atividade parlamentar,08.513.041/0001-93,Rodovias e Vias Editora e Produções LTDA - ME,224,23/12/2015,Despesas com serviços técnicos de comunicação ...,85000.0
192789,2016,12,SÉRGIO PETECÃO,Divulgação da atividade parlamentar,12.974.269/0001-58,GADELHA E SILVEIRA EMPRESA DE COMUNICAÇÃO LTDA,140,23/12/2016,NaN,80000.0
192606,2016,12,ROSE DE FREITAS,Divulgação da atividade parlamentar,05.461.408/0001-84,Grafitusa S/A,32577,31/03/2017,NaN,79680.0


### Solicitação para investigar mais a fundo

Davi Alcolombre, valores altos recorrentes com a mesma empresa, faltando detalhamento de alguns registros

In [62]:
df_divul.loc[201299]

ANO                                                   2017
MES                                                     12
SENADOR                                    DAVI ALCOLUMBRE
TIPO_DESPESA           Divulgação da atividade parlamentar
CNPJ_CPF                                29.337.655/0001-31
FORNECEDOR           Start print Comunicação Visual Eireli
DOCUMENTO                                               36
DATA                                            15/03/2018
DETALHAMENTO                                           NaN
VALOR_REEMBOLSADO                                 270000.0
Name: 201299, dtype: object

### Top 5 parlamentares para serem investigados

In [64]:
df_divul_investigacao = df_divul.groupby(["SENADOR"])["VALOR_REEMBOLSADO"].sum().sort_values(ascending=False)
df_divul_investigacao = pd.DataFrame(df_divul_investigacao, columns=["VALOR_REEMBOLSADO"])
df_divul_investigacao

,VALOR_REEMBOLSADO
SENADOR,
VALDIR RAUPP,1398246.15
SÉRGIO PETECÃO,1353410.69
VANESSA GRAZZIOTIN,1076650.34
IVO CASSOL,975047.01
ÂNGELA PORTELA,857662.16
...,...
TIÃO VIANA,413.00
IRAJÁ,214.02
JARBAS VASCONCELOS,156.70


In [66]:
df_divul_null = df_seg.loc[:, df_divul.columns != "SENADOR"].isnull().groupby(df_seg["SENADOR"]).sum()
df_divul_null["TOTAL_NULO"] = df_divul_null.sum(axis=1).sort_values(ascending=False)
df_divul_null.drop(["ANO", "MES", "TIPO_DESPESA", "CNPJ_CPF", "FORNECEDOR", "DOCUMENTO", "DATA",
                 "DETALHAMENTO", "VALOR_REEMBOLSADO"], axis=1, inplace=True)
df_divul_null.sort_values("TOTAL_NULO", ascending=False, inplace=True)
df_divul_null

,TOTAL_NULO
SENADOR,
JOSÉ AGRIPINO,78
GLADSON CAMELI,69
FERNANDO COLLOR,56
LOBÃO FILHO,38
FÁTIMA BEZERRA,29
CLÉSIO ANDRADE,27
TELMÁRIO MOTA,25
MAILZA GOMES,13
RONALDO CAIADO,10


In [67]:
df_divul_investigacao_final = pd.merge(df_divul_investigacao, df_divul_null, on="SENADOR").sort_values(["VALOR_REEMBOLSADO", "TOTAL_NULO"], ascending=[False,False])
df_divul_investigacao_final.describe()

,VALOR_REEMBOLSADO,TOTAL_NULO
count,4.500000e+01,45.000000
mean,2.266024e+05,9.711111
std,2.798994e+05,17.258229
min,1.567000e+02,0.000000
25%,3.673400e+04,0.000000
50%,1.203180e+05,5.000000
75%,3.383670e+05,9.000000
max,1.353411e+06,78.000000


In [68]:
df_divul_investigacao_final

,VALOR_REEMBOLSADO,TOTAL_NULO
SENADOR,,
SÉRGIO PETECÃO,1353410.69,7
DAVI ALCOLUMBRE,767375.60,3
PAULO PAIM,757952.37,4
MOZARILDO CAVALCANTI,729352.75,6
ROMERO JUCÁ,571509.90,9
WALDEMIR MOKA,557456.37,0
JOÃO CAPIBERIBE,547276.00,0
FERNANDO COLLOR,518385.90,56
LÚCIA VÂNIA,427242.66,7


Deputados escolhidos:

SÉRGIO PETECÃO - ticket bem acima da média

FERNANDO COLLOR - valor na média mas muitos dados faltantes

GLADSON CAMELI - muitos dados faltantes

JOSÉ AGRIPINO - muitos dados faltantes

FÁTIMA BEZERRA - muitos dados faltantes

In [70]:
df_seg_investigacao_final.loc["MARA GABRILLI"]

VALOR_REEMBOLSADO    6682.0
TOTAL_NULO              0.0
Name: MARA GABRILLI, dtype: float64

Mara Gabrilli não é o parlamentar que menos gastou na categoria mas seguindo a ordem crescente, é o primeiro que menos utilizou a cota parlamentar a ter informado todos os dados do reembolso